In [1]:
import jax
import jax.numpy as jnp
import jax.random as jr
import equinox as eqx 
import optax
import matplotlib.pyplot as plt

from gstools import SRF, Gaussian
from gstools.random import MasterRNG
from utils_fdm import *

In [ ]:
def Matrix(dose: float, apply: bool, params: dict, dx: float, dt: float, size: int):

    LAMBDA = (params['D'] * dt) / dx**2
    GAMMA_1 = 1 + (R(dose, apply, params) * dt) / 2 + LAMBDA 
    GAMMA_2 = 1 - (R(dose, apply, params) * dt) / 2 - LAMBDA 

    A = np.zeros((size, size))
    B = np.zeros((size, size))

    for i in range(1, size-1):
        A[i,i] = GAMMA_1
        A[i,i+1] = -LAMBDA / 2
        A[i,i-1] = -LAMBDA / 2

        B[i,i]  = GAMMA_2
        B[i,i+1] = LAMBDA / 2
        B[i,i-1] = LAMBDA / 2

    A[0, 0] = - 3 / (4 * dx)
    A[0, 1] = 1 / dx
    A[0, 2] = -1 / (4 * dx)

    A[-1, -1] = 3 / (4 * dx)
    A[-1, -2] = -1 / dx
    A[-1, -3] = 1 / (4 * dx)

    B[0, 0] = 3 / (4 * dx)
    B[0, 1] = -1 / dx
    B[0, 2] = 1 / (4 * dx)

    B[-1, -1] = -3 / (4 * dx)
    B[-1, -2] = 1 / dx
    B[-1, -3] = -1 / (4 * dx)

    return A, B